In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
from huggingface_hub import login
import pandas as pd
df1 = pd.read_csv('NAICS_data.csv')

login(token="")

tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModel.from_pretrained('distilbert-base-uncased')

def generate_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Using mean pooling
    return embeddings

df1['embeddings'] = df1['INDEX ITEM DESCRIPTION'].apply(lambda x: generate_embeddings(x).detach().numpy())



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [3]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 73.3 MB/s eta 0:00:00


In [40]:
import faiss
import numpy as np

embedding_matrix = np.vstack(df1['embeddings'].values)


index = faiss.IndexFlatL2(embedding_matrix.shape[1])
faiss.normalize_L2(embedding_matrix)
index.add(embedding_matrix)

faiss.write_index(index, 'naics_index.faiss')


In [5]:
query = "A startup developing cloud-based project management software."

query_embedding = generate_embeddings(query).detach().numpy()

# Searching the FAISS index for the top 5 most similar industries
k = 5
D, I = index.search(query_embedding, k)

# Retrieve the top-k similar industries
top_k_results = df1.iloc[I[0]]
print(top_k_results[['NAICS12', 'INDEX ITEM DESCRIPTION']])


      NAICS12                             INDEX ITEM DESCRIPTION
16176  541611              Business start-up consulting services
16199  541613        New product development consulting services
16153  541512  CAM (computer-aided manufacturing) systems int...
16152  541512  CAE (computer-aided engineering) systems integ...
16160  541512  Computer-aided engineering (CAE) systems integ...


In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

t5_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
t5_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [31]:
prompt = f"""
Business Description: A startup is developing cloud-based project management software.

Choose the most appropriate industry description from the following options and provide the description:
{top_k_results['INDEX ITEM DESCRIPTION'].values[0]}
{top_k_results['INDEX ITEM DESCRIPTION'].values[1]}
{top_k_results['INDEX ITEM DESCRIPTION'].values[2]}
{top_k_results['INDEX ITEM DESCRIPTION'].values[3]}
{top_k_results['INDEX ITEM DESCRIPTION'].values[4]}
"""
input_ids = t5_tokenizer.encode(prompt, return_tensors='pt')
output = t5_model.generate(input_ids, max_length=80)
prediction = t5_tokenizer.decode(output[0], skip_special_tokens=True)
print(prediction)

Business start-up consulting services


Retail Industry

In [36]:
query = "An online platform for fashion retail and e-commerce."

query_embedding = generate_embeddings(query).detach().numpy()

k = 5
D, I = index.search(query_embedding, k)

# Retrieve the top-k similar industries
top_k_results = df1.iloc[I[0]]
print(top_k_results[['NAICS12', 'INDEX ITEM DESCRIPTION']])


      NAICS12                             INDEX ITEM DESCRIPTION
15116  519130  Financial magazine and periodical publishers, ...
15103  519130  Business directory publishers, exclusively on ...
15149  519130  Professional magazine and periodical publisher...
14077  448120        Apparel stores, women's and girls' clothing
14244  454111   Business to Consumer retail sales Internet sites


In [39]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

t5_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
t5_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
prompt = f"""
Business Description: An online platform for fashion retail and e-commerce.

Choose the most appropriate industry description from the following options and provide the description:
{top_k_results['INDEX ITEM DESCRIPTION'].values[0]}
{top_k_results['INDEX ITEM DESCRIPTION'].values[1]}
{top_k_results['INDEX ITEM DESCRIPTION'].values[2]}
{top_k_results['INDEX ITEM DESCRIPTION'].values[3]}
{top_k_results['INDEX ITEM DESCRIPTION'].values[4]}
"""
input_ids = t5_tokenizer.encode(prompt, return_tensors='pt')
output = t5_model.generate(input_ids, max_length=150)
prediction = t5_tokenizer.decode(output[0], skip_special_tokens=True)
print(prediction)

Business to Consumer retail sales
